# Vectorization and Array Operations with NumPy

### Written for the CBC Workshop (May 2024)

#### John Stachurski

[NumPy](https://numpy.org/) is the standard library for numerical array operations in Python.

This notebook contains a very quick introduction to NumPy.

(Although the syntax and some reference concepts differ, the basic framework is similar to Matlab.)

We use the following imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## NumPy arrays

Let's review the basics of NumPy arrays.

### Creating arrays

Here are a few ways to create arrays:

In [ ]:
a = (10.0, 20.0)
print(type(a))
a = np.array(a)   # Create array from Python tuple
type(a)

In [ ]:
a = np.array((10, 20), dtype='float64')  # Specify data type -- must be homogeneous
a

In [ ]:
a = np.linspace(0, 10, 5)
a

In [ ]:
a = np.ones(3)
a

In [ ]:
a = np.zeros(3)
a

In [ ]:
a = np.random.randn(4)
a

In [ ]:
a = np.random.randn(2, 2)
a

In [ ]:
b = np.zeros_like(a)
b

### Reshaping

In [ ]:
a = np.random.randn(2, 2)
a

In [ ]:
a.shape

In [ ]:
np.reshape(a, (1, 4))

In [ ]:
np.reshape(a, (4, 1))

### Array operations

Standard arithmetic operators are pointwise:

In [ ]:
a

In [ ]:
b

In [ ]:
a + b

In [ ]:
a * b  # pointwise multiplication

### Matrix multiplication

In [ ]:
a @ b  

In [ ]:
np.ones(3) @ np.zeros(3)  # inner product

### Reductions

There are various functions for acting on arrays, such as

In [ ]:
np.mean(a)

In [ ]:
np.sum(a)

These operations have an equivalent OOP syntax, as in

In [ ]:
a.mean()

In [ ]:
a.sum()

These operations also work on higher-dimensional arrays:

In [ ]:
a = np.linspace(0, 3, 4).reshape(2, 2)
a

In [ ]:
a.sum(axis=0)  # sum columns

In [ ]:
a.sum(axis=1)  # sum rows

### Broadcasting

When possible, arrays are "streched" across missing dimensions to perform array operations.

For example,

In [ ]:
a = np.zeros((3, 3))
a

In [ ]:
b = np.array((1.0, 2.0, 3.0))
b = np.reshape(b, (1, 3))
b

In [ ]:
a + b

In [ ]:
b = np.reshape(b, (3, 1))
b

In [ ]:
a + b

For more on broadcasting see [this tutorial](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html).

### Ufuncs

Many NumPy functions can act on either scalars or arrays.

When they act on arrays, they act pointwise (element-by-element).

These kinds of functions are called **universal functions** or **ufuncs**.

In [ ]:
np.cos(np.pi)

In [ ]:
a = np.random.choice((0, np.pi), 6).reshape(2, 3)
a

In [ ]:
np.cos(a)

Some user-defined functions will be ufuncs, such as

In [ ]:
def f(x):
    return np.cos(np.sin(x))

In [ ]:
f(a)

But some are not:

In [ ]:
def f(x):
    if x < 0:
        return np.cos(x)
    else:
        return np.sin(x)

In [ ]:
f(a)

If we want to turn this into a vectorized function we can use `np.vectorize`

In [ ]:
f_vec = np.vectorize(f)

Let's test it, and also time it.

In [ ]:
a = np.linspace(0, 1, 10_000_000)
%time f_vec(a)

This is pretty slow.

Here's a version of `f` that uses NumPy functions to create a more efficient ufunc.

In [ ]:
def f(x):
    return np.where(x < 0, np.cos(x), np.sin(x))

In [ ]:
%time f(a)

Moral of the story: Don't use `np.vectorize` unless you have to.

(There are good alternatives, which we will discuss soon.)

### Mutability

NumPy arrays are mutable (can be altered in memory).

In [ ]:
a = np.array((10.0, 20.0))
a

In [ ]:
a[0] = 1

In [ ]:
a

In [ ]:
a[:] = 42

In [ ]:
a

**All names** bound to an array have equal rights.

In [ ]:
a

In [ ]:
b = a  # bind the name b to the same array object

In [ ]:
id(a)

In [ ]:
id(b)

In [ ]:
b[0] = 1_000

In [ ]:
b

In [ ]:
a

## Vectorizing loops

### Accelerating slow loops

In scripting languages, native loops are slow:

In [ ]:
n = 10_000_000
x_vec = np.linspace(0.1, 1.1, n)

Let's say we want to compute the sum of of $\cos(2\pi / x)$ over $x$ in

In [ ]:
%%time
current_sum = 0.0
for x in x_vec:
    current_sum += np.cos(2 * np.pi / x)

The reason is that Python, like most high level languages is dynamically typed.

This means that the type of a variable can freely change.

Moreover, the interpreter doesn't compile the whole program at once, so it doesn't know when types will change.

So the interpreter has to check the type of variables before any operation like addition, comparison, etc.

Hence there's a lot of fixed cost for each such operation

The code runs much faster if we use **vectorized** expressions to avoid explicit loops.

In [ ]:
%%time
np.sum(np.cos(2 * np.pi / x_vec))

Now high level overheads are paid *per array rather than per float*.

### Implict Multithreading


Recent versions of Anaconda are compiled with Intel MKL support, which accelerates NumPy operations.

Watch system resources when you run this code.  

(For example, install `htop` (Linux / Mac), `perfmon` (Windows) or another system load monitor and set it running in another window.)

In [ ]:
n = 20
m = 1000
for i in range(n):
    X = np.random.randn(m, m)
    λ = np.linalg.eigvals(X)

You should see all your cores light up.  With MKL, many matrix operations are automatically parallelized.